# **ETL Items**

<div style="text-align: justify">
En este notebook de Python, se detallarán los pasos necesarios para llevar a cabo un proceso de ETL (Extract, Transform, Load) en un dataframe que denominaremos "items". Los datos que utilizaremos provienen originalmente de un archivo JSON llamado "australian_users_items.json". A lo largo de este documento, se explorarán las etapas de extracción, transformación y carga de datos con el objetivo de preparar el conjunto de datos para su posterior análisis y uso en aplicaciones o modelos de aprendizaje automático.
</div>

## 1. Importación de las librerias

<div style="text-align: justify">
Importar bibliotecas en un ETL es esencial para aprovechar las funcionalidades predefinidas que simplifican tareas como la extracción, transformación y carga de datos. Esto ahorra tiempo y esfuerzo, asegurando un desarrollo eficiente y la creación de procesos ETL más sólidos y confiables.
</div>

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import json
import ast
import funciones_basicas as fb

## 2. Carga de los datos

En estas lineas de código cargaremos los datos y creamos el dataframe de pandas en el que vamos a realizar el proceso de ETL, todo basado en un arhivo llamado *australian_users_review.json*, como presenta una estructura diferente a nuestro primer dataframe, usaremos otro metodo para abirlo

In [2]:
datos_filas = []

with open('Datasets\\australian_users_items.json', 'r', encoding='utf-8') as archivo:  
    for linea in archivo:  
        datos_filas.append(ast.literal_eval(linea))

items = pd.DataFrame(datos_filas)
items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


## 3. Transformaciones

<div style='text-align: justify'>
En esta sección realizaremos las primeras transformaciones, estas transformaciones se realizan como una buena practica y son transversales a cualquier tipo de datos no importa su contenido, posterior a esas transformacioens inspeccionaremos a fondo la información del dataframe y realizaremos todos los procesos correspondientes, partiendo de ahi. 
</div>

In [3]:
## NORMALIZACION COLUMNA ITEMS
items = pd.json_normalize(datos_filas, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )

In [4]:
# Eliminamos los datos donde el tiempo de juego sea 0
items = items[items['playtime_forever'] != 0]

# Resetear índices
items.reset_index(drop=True, inplace=True)

In [5]:
# Crear la columna 'playtime_hours' y convertir 'playtime_forever' de minutos a horas
items.loc[:, 'playtime_hours'] = round(items['playtime_forever'] / 60, 2)
items.drop(columns=['playtime_forever'], inplace=True)

In [6]:
## TRATAMIENTO DE DATOS NULOS Y DUPLICADOS
items.dropna(inplace=True)
items.drop_duplicates(inplace=True)

In [7]:
# Eliminar columnas no necesarias
items.drop(['playtime_2weeks', 'user_url'], axis=1, inplace=True)

In [8]:
# Limpieza y manipulación de texto para la columna item_name
items = items.map(fb.normalizar_texto)
items.head(2)

,item_id,item_name,steam_id,items_count,user_id,playtime_hours
0,10,COUNTER-STRIKE,76561197970982479,277,76561197970982479,0.10
1,30,DAY OF DEFEAT,76561197970982479,277,76561197970982479,0.12


In [9]:
# Cambio de nombres a las columnas
nuevos_nombres = {'item_id':'ID_GAME',
                  'item_name':'NOMBRE_JUEGO',
                  'steam_id':'ID_STEAM',
                  'items_count':'CANT_JUEGOS',
                  'user_id':'ID_USUARIO',
                  'playtime_hours':'HORAS_DE_JUEGO'}

items = items.rename(columns=nuevos_nombres)
items.head(3)

,ID_GAME,NOMBRE_JUEGO,ID_STEAM,CANT_JUEGOS,ID_USUARIO,HORAS_DE_JUEGO
0,10,COUNTER-STRIKE,76561197970982479,277,76561197970982479,0.10
1,30,DAY OF DEFEAT,76561197970982479,277,76561197970982479,0.12
2,300,DAY OF DEFEAT: SOURCE,76561197970982479,277,76561197970982479,78.88


## 4. Guardamos el archivo

<div style="text-align: justify">
Despues de realizadas todas las transformaciones necesarias para manejar el dataframe en futuros analisis, procedemos a guardar el documento en un formato tipo parquet para que su lectura y escritura sea más sencilla
</div>

In [10]:
fb.guardar_datos(items, "PARQUET\\ITEMS.parquet")